In [1]:
%load_ext autoreload  
%autoreload 2  
  
import random  
import numpy as np  
import torch  
import torch.nn as nn  
from torch.autograd import Function  
import optuna  
from modules.utils import evaluate_model  
import matplotlib.pyplot as plt  
from braindecode.models import EEGInceptionMI, EEGSimpleConv, MSVTNet, FBCNet, ATCNet
from braindecode import EEGClassifier  
import torch.optim as optim  
  
# dataset related  
from modules import CompetitionDataset, load_combined_moabb_data  
from torch.utils.data import DataLoader, TensorDataset  
from moabb.datasets import BNCI2014_001, PhysionetMI, Weibo2014, Cho2017  # 250 hz  
from braindecode.datasets import MOABBDataset, create_from_X_y  
from braindecode.preprocessing import preprocess, Preprocessor, exponential_moving_standardize, filterbank  
from skorch.helper import predefined_split  
from braindecode.datasets.base import BaseConcatDataset  
import random
import mne
import numpy as np
  
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
device  

/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cpu')

In [2]:
batch_size = 64
# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [ ]:
moabb_train_datasets = [
    PhysionetMI(imagined=True),  # 109 subjects
    Weibo2014(),  # 10 subjects, 64 channels
    CompetitionDataset(),
]
train_val = [CompetitionDataset(split="validation")]

# Load combined data # BEST tmin=1, tmax=4, resample=250
eeg_channels = ["Fz", "C3", "Cz", "C4", "Pz"]
X_train, y_train, domain_labels_train, info_train = load_combined_moabb_data(
    datasets=moabb_train_datasets,
    paradigm_config={
        "channels": eeg_channels,
        "tmin": 1.0,
        "tmax": 4.0,
        "resample": 250,
    },
    subjects_per_dataset={
        "PhysionetMI": list(range(1, 50)),
        "Weibo2014": list(range(1, 11)),
        "CompetitionDataset": list(range(1, 31)),
    },
)

# Load combined data
X_val, y_val, domain_labels_val, info_val = load_combined_moabb_data(
    datasets=train_val,
    paradigm_config={
        "channels": eeg_channels,
        "tmin": 1.0,
        "tmax": 4.0,
        "resample": 250,
    },
)

# Best preprocessor
# preprocessors = [
#     Preprocessor('resample', sfreq=250),
#     Preprocessor('filter', l_freq=1, h_freq=40),
#     # Preprocessor('pick_channels', ch_names=["C3", "Cz", "C4", "FC3", "FC4", "CP3", "CP4"]),
#     Preprocessor('set_eeg_reference', ref_channels='average'),  # Common average reference
#     Preprocessor(exponential_moving_standardize, factor_new=1e-3, init_block_size=1000),
#     Preprocessor(filterbank, frequency_bands=[(8, 12), (13, 30)], drop_original_signals=False),  # Mu/Beta bands
# ]


Processing dataset: PhysionetMI
Original subject range: 1 to 49
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']


/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 1 – 4 s (baseline off), ~315 KiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 1 – 4 s (baseline off), ~315 KiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 1 – 4 s (baseline off), ~315 KiB, data loaded,
 'left_hand': 7
 'right_hand': 8>
  warn(f"warnEpochs {epochs}")


Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']


/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 1 – 4 s (baseline off), ~315 KiB, data loaded,
 'left_hand': 7
 'right_hand': 8>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 1 – 4 s (baseline off), ~315 KiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 15 events (all good), 1 – 4 s (baseline off), ~315 KiB, data loaded,
 'left_hand': 8
 'right_hand': 7>
  warn(f"warnEpochs {epochs}")


Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['feet', 'hands', 'rest']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']
Used Annotations descriptions: ['left_hand', 'rest', 'right_hand']


...

Adjusted subject range: (50, 59)
Number of trials: 1580
Number of subjects: 10

Processing dataset: CompetitionDataset
Original subject range: 1 to 30


No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
No stim channel nor annotations found, skipping setting annotations.
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 4 s (baseline off), ~302 KiB, data loaded,
 'left_hand': 6
 'right_hand': 4>
  warn(f"warnEpochs {epochs}")
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 4 

Adjusted subject range: (60, 89)
Number of trials: 2400
Number of subjects: 30

=== COMBINED DATASET SUMMARY ===
Total trials: 6180
Feature shape: (6180, 5, 750)
Class distribution: [3086 3094]
Subject range: 1 to 89
Total unique subjects: 89

Processing dataset: CompetitionDataset
Original subject range: 1 to 30


/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 4 s (baseline off), ~302 KiB, data loaded,
 'left_hand': 5
 'right_hand': 5>
  warn(f"warnEpochs {epochs}")
No stim channel nor annotations found, skipping setting annotations.
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 4 s (baseline off), ~302 KiB, data loaded,
 'left_hand': 6
 'right_hand': 4>
  warn(f"warnEpochs {epochs}")
No stim channel nor annotations found, skipping setting annotations.
/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 10 events (all good), 1 – 4 s (baseline off), ~302 KiB, data loaded,
 'left_hand': 4
 'right_hand': 6>
  warn(f"warnEpochs {epochs}")
No stim channel nor annotations




WARNING TASK DF EMPTY 6 AT ROW 36 AT SPLIT validation






WARNING TASK DF EMPTY 7 AT ROW 37 AT SPLIT validation






WARNING TASK DF EMPTY 8 AT ROW 38 AT SPLIT validation






WARNING TASK DF EMPTY 9 AT ROW 39 AT SPLIT validation






WARNING TASK DF EMPTY 10 AT ROW 40 AT SPLIT validation






WARNING TASK DF EMPTY 11 AT ROW 41 AT SPLIT validation






WARNING TASK DF EMPTY 12 AT ROW 42 AT SPLIT validation






WARNING TASK DF EMPTY 13 AT ROW 43 AT SPLIT validation






WARNING TASK DF EMPTY 14 AT ROW 44 AT SPLIT validation






WARNING TASK DF EMPTY 15 AT ROW 45 AT SPLIT validation






WARNING TASK DF EMPTY 16 AT ROW 46 AT SPLIT validation






WARNING TASK DF EMPTY 17 AT ROW 47 AT SPLIT validation






WARNING TASK DF EMPTY 18 AT ROW 48 AT SPLIT validation






WARNING TASK DF EMPTY 19 AT ROW 49 AT SPLIT validation






WARNING TASK DF EMPTY 20 AT ROW 50 AT SPLIT validation






WARNING TASK DF EMPTY 21 AT ROW 51 AT SPLIT validation






WARNING TASK DF EMPTY 22 

In [ ]:
def preprocess_numpy(
    X,  # shape (n_trials, n_chans, n_times)
    sfreq=250,
    l_freq=1.0,
    h_freq=40.0,
    ref="average",
    bands=[(8, 12), (13, 30)],
    ems_alpha=1e-3,
    ems_init_block=1000,
    ems_eps=1e-4,
):
    # a) Bandpass filter the full signal
    X_filt = np.stack([mne.filter.filter_data(trial, sfreq, l_freq, h_freq, verbose=False) for trial in X], axis=0)

    # b) Common‑average reference
    if ref == "average":
        X_filt = X_filt - X_filt.mean(axis=1, keepdims=True)
    else:
        X_filt = X_filt

    # c) Exponential‑moving standardize per trial & channel
    #    m_t and v_t initialized from first ems_init_block samples
    n_trials, n_chans, n_times = X_filt.shape
    X_filt = np.zeros_like(X_filt)
    for i in range(n_trials):
        trial = X_filt[i]
        # initialize m, v
        m = trial[:, :ems_init_block].mean(axis=1, keepdims=True)  # (n_chans,1)
        v = trial[:, :ems_init_block].var(axis=1, keepdims=True)  # (n_chans,1)
        for t in range(n_times):
            x_t = trial[:, t : t + 1]  # (n_chans,1)
            # update
            m = (1 - ems_alpha) * m + ems_alpha * x_t
            v = (1 - ems_alpha) * v + ems_alpha * ((x_t - m) ** 2)
            # standardize
            X_filt[i, :, t] = ((x_t - m) / np.sqrt(v + ems_eps)).ravel()

    # d) Filterbank: for each band, bandpass the EMS‑normalized signals
    # fb_list = []
    # for low, high in bands:
    #     fb = np.stack([mne.filter.filter_data(trial, sfreq, low, high, verbose=False) for trial in X_filt], axis=0)
    #     fb_list.append(fb)
    # X_filt = np.concatenate(fb_list, axis=1)  # now shape (n_trials, n_chans*len(bands), n_times)

    return X_filt


# for some reason i can't understand preprocessing kills model accuracy

# X_train = preprocess_numpy(
#     X_train,
#     sfreq=250,
#     l_freq=1,
#     h_freq=40,
#     ref="average",
#     bands=[(8, 12), (13, 30)],
#     ems_alpha=1e-3,
#     ems_init_block=1000,
# )
# X_val = preprocess_numpy(
#     X_val,
#     sfreq=250,
#     l_freq=1,
#     h_freq=40,
#     ref="average",
#     bands=[(8, 12), (13, 30)],
#     ems_alpha=1e-3,
#     ems_init_block=1000,
# )

/tmp/ipykernel_240612/27300938.py:13: RuntimeWarning: filter_length (825) is longer than the signal (750), distortion is likely. Reduce filter length or filter a longer signal.
  X_filt = np.stack([mne.filter.filter_data(trial, sfreq, l_freq, h_freq, verbose=False) for trial in X], axis=0)
/tmp/ipykernel_240612/27300938.py:13: RuntimeWarning: filter_length (825) is longer than the signal (750), distortion is likely. Reduce filter length or filter a longer signal.
  X_filt = np.stack([mne.filter.filter_data(trial, sfreq, l_freq, h_freq, verbose=False) for trial in X], axis=0)
/tmp/ipykernel_240612/27300938.py:13: RuntimeWarning: filter_length (825) is longer than the signal (750), distortion is likely. Reduce filter length or filter a longer signal.
  X_filt = np.stack([mne.filter.filter_data(trial, sfreq, l_freq, h_freq, verbose=False) for trial in X], axis=0)
/tmp/ipykernel_240612/27300938.py:13: RuntimeWarning: filter_length (825) is longer than the signal (750), distortion is likely

In [4]:
X_train = torch.from_numpy(X_train).float()  # FloatTensor of shape (N, C, T)
y_train = torch.from_numpy(y_train).long()  # LongTensor of shape (N, 2)

train_dataset = TensorDataset(X_train, y_train)

X_val_t = torch.from_numpy(X_val).float()
y_val_t = torch.from_numpy(y_val).long()

val_dataset = TensorDataset(X_val_t, y_val_t)


In [14]:
model = EEGInceptionMI(
    n_chans=5,  # Number of channels
    n_outputs=2,  # Number of classes (adjust based on your task)
    input_window_seconds=3.0,  # best is 3-second windows 1->4
    sfreq=250,
    n_filters=12,  # Optimized parameter
    n_convs=5,
    kernel_unit_s=0.1,
)

# Setup classifier with skorch wrapper
clf = EEGClassifier(
    model,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    optimizer__lr=0.001,
    batch_size=64,
    max_epochs=100,
    train_split=predefined_split(val_dataset), 
    device="cuda" if torch.cuda.is_available() else "cpu",
    verbose=2,
    callbacks=['accuracy'],
)

clf.fit(X_train, y_train)

  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      dur
-------  ----------------  ------------  -----------  ----------------  ------------  -------
      1            0.5088        0.6969       0.5600            0.5600        0.8803  58.1216
      2            0.4912        0.6974       0.4400            0.4400        0.7037  49.9941


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=================================================================================================================================================================
  Layer (type (var_name):depth-idx)                            Input Shape               Output Shape              Param #                   Kernel Shape
  ================================================================================================================================================================
  EEGInceptionMI (EEGInceptionMI)                              [1, 5, 750]               [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1                                 [1, 5, 750]               [1, 5, 750, 1]            --                        --
  ├─Rearrange (dimshuffle): 1-2                                [1, 5, 750, 1]            [1, 5, 1, 750]            --                        --
  ├─_ResidualModuleMI (residual_block_1): 1-3                  [1, 5, 1, 750]            [1, 72, 1, 750]           --                        --
  │    └─Conv2d (conv): 2-1                                    [1, 5, 1, 750]            [1, 72, 1, 750]           432                       [1, 1]
  │    └─BatchNorm2d (bn): 2-2                                 [1, 72, 1, 750]           [1, 72, 1, 750]           144                       --
  │    └─ReLU (activation): 2-3                                [1, 72, 1, 750]           [1, 72, 1, 750]           --                        --
  ├─_InceptionModuleMI (initial_inception_module): 1-4         [1, 5, 1, 750]            [1, 72, 1, 750]           --                        --
  │    └─Conv2d (bottleneck): 2-4                              [1, 5, 1, 750]            [1, 12, 1, 750]           72                        [1, 1]
  │    └─ModuleList (conv_list): 2-5                           --                        --                        --                        --
  │    │    └─Conv2d (0): 3-1                                  [1, 12, 1, 750]           [1, 12, 1, 750]           3,612                     [1, 25]
  │    │    └─Conv2d (1): 3-2                                  [1, 12, 1, 750]           [1, 12, 1, 750]           10,812                    [1, 75]
  │    │    └─Conv2d (2): 3-3                                  [1, 12, 1, 750]           [1, 12, 1, 750]           18,012                    [1, 125]
  │    │    └─Conv2d (3): 3-4                                  [1, 12, 1, 750]           [1, 12, 1, 750]           25,212                    [1, 175]
  │    │    └─Conv2d (4): 3-5                                  [1, 12, 1, 750]           [1, 12, 1, 750]           32,412                    [1, 225]
  │    └─MaxPool2d (pooling): 2-6                              [1, 5, 1, 750]            [1, 5, 1, 750]            --                        [1, 25]
  │    └─Conv2d (pooling_conv): 2-7                            [1, 5, 1, 750]            [1, 12, 1, 750]           72                        [1, 1]
  │    └─BatchNorm2d (bn): 2-8                                 [1, 72, 1, 750]           [1, 72, 1, 750]           144                       --
  │    └─ReLU (activation): 2-9                                [1, 72, 1, 750]           [1, 72, 1, 750]           --                        --
  ├─ModuleList (intermediate_inception_modules_1): 1-5         --                        --                        --                        --
  │    └─_InceptionModuleMI (0): 2-10                          [1, 72, 1, 750]           [1, 72, 1, 750]           --                        --
  │    │    └─Conv2d (bottleneck): 3-6                         [1, 72, 1, 750]           [1, 12, 1, 750]           876                       [1, 1]
  │    │    └─ModuleList (conv_list): 3-7                      --                        --                        90,060                    --
  │    │    └─MaxPool2d (pooling): 3-8                         [1, 72, 1, 750]           [

In [5]:
model = EEGSimpleConv(
    n_chans=5,
    n_outputs=2,  # Left/right hand motor imagery
    sfreq=80,  # Optimal resampling frequency for this model is 80hz
    feature_maps=96,  # Within recommended range [64-144]
    n_convs=2,  # For cross-subject: [2-4]
    kernel_size=8,  # For cross-subject: [5-8]
    resampling_freq=80,
)

clf = EEGClassifier(
    model,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    optimizer__lr=0.0001,
    batch_size=64,
    max_epochs=100,
    train_split=predefined_split(val_dataset),
    device="cuda" if torch.cuda.is_available() else "cpu",
    verbose=2,
    callbacks=["accuracy"],
)

clf.fit(X_train, y_train)

  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss      dur
-------  ----------------  ------------  -----------  ----------------  ------------  -------
      1            0.5087        0.6977       0.4600            0.4600        0.7295  47.8816
      2            0.5097        0.6916       0.4600            0.4600        0.7574  30.2517
      3            0.5487        0.6894       0.4200            0.4200        0.7637  41.2819
      4            0.5398        0.6851       0.5600            0.5600        0.7176  39.4866
      5            0.5688        0.6827       0.4400            0.4400        1.0256  39.8291
      6            0.5536        0.6766       0.4400            0.4400        0.9940  43.6035
      7            0.5587        0.6732       0.4800            0.4800        0.9058  45.2907
      8            0.5791        0.6680       0.5600            0.5600        0.7234  42.4140
      9            0.5971        0.6617       0.4000        

ValueError: n_times could not be inferred. Either specify n_times or input_window_seconds and sfreq.

ValueError: n_times could not be inferred. Either specify n_times or input_window_seconds and sfreq.

ValueError: n_times could not be inferred. Either specify n_times or input_window_seconds and sfreq.

In [8]:
print(X_train.shape, y_train.shape)

model = ATCNet(
    n_chans=5,
    n_outputs=2,  # Left/right hand motor imagery
    input_window_seconds=3.0,
    sfreq=250,
    n_windows=5,
    att_head_dim=8,
    att_num_heads=2,
    tcn_depth=2,
    tcn_kernel_size=4,
    tcn_n_filters=32,
)

clf = EEGClassifier(
    model,
    criterion=torch.nn.CrossEntropyLoss,
    optimizer=torch.optim.Adam,
    optimizer__lr=0.0005,  # Lower learning rate for attention models
    batch_size=32,  # Smaller batch size for memory efficiency
    max_epochs=100,
    train_split=predefined_split(val_dataset),
    device="cuda" if torch.cuda.is_available() else "cpu",
    verbose=2,
    callbacks=["accuracy"],
)

clf.fit(X_train, y_train)

torch.Size([4080, 5, 750]) torch.Size([4080])


/home/zeyadcode/.pyenv/versions/icmtc_venv/lib/python3.12/site-packages/torch/nn/modules/conv.py:549: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at /pytorch/aten/src/ATen/native/Convolution.cpp:1036.)
  return F.conv2d(


  epoch    train_accuracy    train_loss    valid_acc    valid_accuracy    valid_loss     dur
-------  ----------------  ------------  -----------  ----------------  ------------  ------
      1            0.5002        0.7375       0.4400            0.4400        0.7243  6.5850
      2            0.5002        0.7222       0.4400            0.4400        0.7165  6.4555
      3            0.4998        0.7153       0.5600            0.5600        0.6907  6.4192
      4            0.4998        0.7109       0.5600            0.5600        0.6859  6.4900


<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=================================================================================================================================================================
  Layer (type (var_name):depth-idx)                            Input Shape               Output Shape              Param #                   Kernel Shape
  ================================================================================================================================================================
  ATCNet (ATCNet)                                              [1, 5, 750]               [1, 2]                    --                        --
  ├─Ensure4d (ensuredims): 1-1                                 [1, 5, 750]               [1, 5, 750, 1]            --                        --
  ├─Rearrange (dimshuffle): 1-2                                [1, 5, 750, 1]            [1, 1, 750, 5]            --                        --
  ├─_ConvBlock (conv_block): 1-3                               [1, 1, 750, 5]            [1, 32, 13, 1]            --                        --
  │    └─Conv2d (conv1): 2-1                                   [1, 1, 750, 5]            [1, 16, 750, 5]           1,024                     [64, 1]
  │    └─BatchNorm2d (bn1): 2-2                                [1, 16, 750, 5]           [1, 16, 750, 5]           32                        --
  │    └─Conv2d (conv2): 2-3                                   [1, 16, 750, 5]           [1, 32, 750, 1]           160                       [1, 5]
  │    └─BatchNorm2d (bn2): 2-4                                [1, 32, 750, 1]           [1, 32, 750, 1]           64                        --
  │    └─ELU (activation2): 2-5                                [1, 32, 750, 1]           [1, 32, 750, 1]           --                        --
  │    └─AvgPool2d (pool2): 2-6                                [1, 32, 750, 1]           [1, 32, 93, 1]            --                        [8, 1]
  │    └─Dropout2d (drop2): 2-7                                [1, 32, 93, 1]            [1, 32, 93, 1]            --                        --
  │    └─Conv2d (conv3): 2-8                                   [1, 32, 93, 1]            [1, 32, 93, 1]            16,384                    [16, 1]
  │    └─BatchNorm2d (bn3): 2-9                                [1, 32, 93, 1]            [1, 32, 93, 1]            64                        --
  │    └─ELU (activation3): 2-10                               [1, 32, 93, 1]            [1, 32, 93, 1]            --                        --
  │    └─AvgPool2d (pool3): 2-11                               [1, 32, 93, 1]            [1, 32, 13, 1]            --                        [7, 1]
  │    └─Dropout2d (drop3): 2-12                               [1, 32, 13, 1]            [1, 32, 13, 1]            --                        --
  ├─ModuleList (attention_blocks): 1-16                        --                        --                        (recursive)               --
  │    └─_AttentionBlock (0): 2-13                             [1, 32, 9]                [1, 32, 9]                --                        --
  │    │    └─Rearrange (dimshuffle): 3-1                      [1, 32, 9]                [1, 9, 32]                --                        --
  │    │    └─LayerNorm (ln): 3-2                              [1, 9, 32]                [1, 9, 32]                64                        --
  │    │    └─_MHA (mha): 3-3                                  [1, 9, 32]                [1, 9, 32]                2,128                     --
  │    │    └─Dropout (drop): 3-4                              [1, 9, 32]                [1, 9, 32]                --                        --
  │    │    └─Rearrange (dimshuffle): 3-5                      [1, 9, 32]                [1, 32, 9]                --                        --
  ├─ModuleList (temporal_conv_nets): 1-17                      --                        --                        (r